# Dataset de Airbnb NYC

In [1]:
import pandas as pd
import numpy as np

### La siguiente celda  guarda cada hoja del archivo excel en un dataframe

In [6]:
df1 = pd.read_excel("C:/Users/migest/OneDrive - CTAG/Documentos/Formaciones/Machine learning/ML-CTAG/Pandas/Ejercicios/Datasets/Airbnb_NYC.xlsx",sheet_name = "NYC1")
df2 = pd.read_excel("C:/Users/migest/OneDrive - CTAG/Documentos/Formaciones/Machine learning/ML-CTAG/Pandas/Ejercicios/Datasets/Airbnb_NYC.xlsx",sheet_name = "NYC2")

**1. Une los dos dataframe anteriores por la columna "id"**

In [15]:
df = pd.merge(df1, df2, on='id')
df.head(10)

,id,name,host_id,host_name,municipio,barrio,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1.0,9.0,2018-10-19,0.21,6.0,365.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1.0,45.0,2019-05-21,0.38,2.0,355.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3.0,0.0,NaT,NaN,1.0,365.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1.0,270.0,2019-07-05,4.64,1.0,194.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10.0,9.0,2018-11-19,0.10,1.0,0.0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200.0,3.0,74.0,2019-06-22,0.59,1.0,129.0
6,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60.0,45.0,49.0,2017-10-05,0.40,1.0,0.0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79.0,2.0,430.0,2019-06-24,3.47,1.0,220.0
8,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79.0,2.0,118.0,2017-07-21,0.99,1.0,0.0
9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150.0,1.0,160.0,2019-06-09,1.33,4.0,188.0


In [39]:
#Crear nueva columna con logaritmo del precio. Esto es simplemente un ejemplo para el apartado de normalización
df['new_price'] = np.log(df['price'])
df['new_price'] = df['price'].apply(lambda x: np.log(x))

c:\Anaconda3\envs\ctag_formacion\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\migest\AppData\Local\Temp\ipykernel_25252\2993153349.py:3: RuntimeWarning: divide by zero encountered in log
  df['new_price'] = df['price'].apply(lambda x: np.log(x))


**2.	¿Si quiero hospedarme en Manhattan una sola noche y para elegir el Airbnb me guio por el que tenga el mayor número de *review* donde me debería de hospedar?**

In [16]:
df.loc[(df['minimum_nights']>1) & (df['municipio']=='Manhattan'),:].sort_values('number_of_reviews', ascending=False).head(1)

,id,name,host_id,host_name,municipio,barrio,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
1879,834190,Manhattan Lux Loft.Like.Love.Lots.Look !,2369681,Carol,Manhattan,Lower East Side,40.71921,-73.99116,Private room,99.0,2.0,540.0,2019-07-06,6.95,1.0,179.0


**3.	 Busco un Airbnb en el municipio donde el precio promedio de alquiler sea el más bajo y la cantidad de *review* por mes sea el mayor. ¿Cuáles son las 3 habitaciones que mejor se adaptan a estos criterios?**

In [19]:
dfmedia_precio = df['price'].mean()
dfmedia_precio

152.7206871868289

In [29]:
# Obtener el municipio con el precio medio menor:
# df.groupby('municipio')['price'].mean().idxmin()
df.loc[(df['municipio']==df.groupby('municipio')['price'].mean().idxmin()),:].sort_values("reviews_per_month", ascending=False).head(3)['name']

23371                       IDEALLY LOCATED COZY COTTAGE
41945    Peaceful 2 Bedroom apartment in a private house
42252         TW #6 Private Suite 2nd Fl.  1 to 4 Guests
Name: name, dtype: object

**4.	Quiero alquilar un Airbnb completo por un total de 200 días, en Brooklyn o Manhattan; no quiero pagar más de 150 dólares por noche, ¿Cuáles son las 5 opciones más economicas que me ofrece Airbnb?**

In [37]:
df.loc[(df['room_type']== 'Entire home/apt') & 
       (df['availability_365']>=200) &
       (df['price']>= 150) &
       (df['municipio'].isin(['Brooklyn', "Manhattan"]))]

,id,name,host_id,host_name,municipio,barrio,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1.0,45.0,2019-05-21,0.38,2.0,355.0
16,7097,Perfect for Your Parents + Garden,17571,Jane,Brooklyn,Fort Greene,40.69169,-73.97185,Entire home/apt,215.0,2.0,198.0,2019-06-28,1.72,1.0,321.0
19,7750,Huge 2 BR Upper East Cental Park,17985,Sing,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190.0,7.0,0.0,NaT,NaN,2.0,249.0
44,12343,BEST BET IN HARLEM,47727,Earl,Manhattan,Harlem,40.81175,-73.94478,Entire home/apt,150.0,7.0,97.0,2019-06-13,0.84,1.0,309.0
51,14287,Cozy 1BD on Central Park West in New York City,56094,Joya,Manhattan,Upper West Side,40.78635,-73.97008,Entire home/apt,151.0,2.0,73.0,2019-07-07,0.63,1.0,302.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48826,36449743,Brooklyn's finest,66084717,Tim,Brooklyn,East Flatbush,40.65170,-73.92580,Entire home/apt,200.0,3.0,0.0,NaT,NaN,2.0,206.0
48844,36453952,West Village Studio on quiet cobblestone street,115491896,Will,Manhattan,West Village,40.73620,-74.00827,Entire home/apt,205.0,1.0,0.0,NaT,NaN,1.0,365.0
48856,36457700,"Large 3 bed, 2 bath , garden , bbq , all you need",66993395,Thomas,Brooklyn,Bedford-Stuyvesant,40.68886,-73.92879,Entire home/apt,345.0,4.0,0.0,NaT,NaN,3.0,354.0
48873,36477307,Brooklyn paradise,241945355,Clement & Rose,Brooklyn,Flatlands,40.63116,-73.92616,Entire home/apt,170.0,1.0,0.0,NaT,NaN,2.0,363.0


**5.	En un nuevo *Dataframe* muestra por municipio: el número de habitaciones tipo individual, casa o departamento entero y compartido; también muestra en otra columna el precio promedio de todas las habitaciones por municipio.**

**6.	Crea una nueva columna con el rating de acuerdo a la siguiente formula, donde las variables son: puntuación del municipio, número de reviews y disponibilidad de la habitación al año.**

**Formula: Rating = (Puntuación de ciudad) + (número de review/ 629) + (disponibilidad / 365).**

La puntuación para cada ciudad es la siguiente {"Manhattan":10/10,"Brooklyn":8/10,"Queens":6/10,"Staten Island":4/10,"Bronx":2/10}.

**7. Encuentra las 5 habitaciones de cada municipio con mayor rating**

**8. Crea un dataframe con la cantidad de reviews de cada municipio de acuerdo a tipo de habitación**

**9. Encuentra los 10 barrios con mayor precio por habitación individual**

**10. Encuentra el barrio con mejor rating de cada municipio y después seleccionar los 2 barrios con el menor precio por "dep/casa completo".**